In [133]:
import pandas as pd
import numpy as np

In [134]:
df_complaints =pd.read_excel(r'C:\Medline\2. CPM\2. US Complaints\US Complaint Data 201901-202311.xlsx',sheet_name=1)

C:\Users\ypeng\AppData\Local\Temp\ipykernel_23068\1521272294.py:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  df_complaints =pd.read_excel(r'C:\Medline\2. CPM\2. US Complaints\US Complaint Data 201901-202311.xlsx',sheet_name=1)


In [135]:
vendor_mapping = pd.read_excel(r'C:\Medline\2. CPM\data\vendor_mapping\Vendor _mapping 2023_v1.xlsx')
vendor_mapping['Vendor Number'] = vendor_mapping['Vendor Number'].map(str)

In [136]:
df_complaints['Notification Created Date'] = pd.to_datetime(df_complaints['Notification Created Date'],format='%Y-%m-%d')

In [137]:
(
    df_complaints
    .query("`If Manufacturing Complaint` == 'Y' and Year.isin([2021,2022,2023])")
    .assign(Quarter =lambda d: d['Notification Created Date'].dt.quarter)
    .pivot_table(
        index = ['Division'],
        columns = ['Year','Quarter'],
        values = ['Notification Number'],
        aggfunc ='count',
        fill_value=0,
        margins=True
    )
)

Notification Number                                                \
Year                    2021                   2022                   2023   
Quarter                    1     2     3    4     1     2     3    4     1   
Division                                                                     
10                       158    71    69   39   109    84    69   60    62   
15                        52    35    28   15    33    34    27    2    61   
18                        19    23    17    3     5     9     9    3     7   
20                         1     7     5    2     1     3     1    0     2   
21                        15    17    16   10    17    19    14   11     9   
22                        80    59    34   35    29    29    34   23    41   
29                         2     0     3    1     3     2     2    0     1   
30                      1050   742   802  814   933   981  1239  729  1083   
32                         0     0     0    0     0     0     0    0    47   
33                         0     2     1    3     6     0     0    0     0   
34                         0     0     0    0     0     0     0    0    37   
35                        98     1     0    0    32    33    12    2     7   
40                         8    10     9   12    12     5    19    9    17   
41                         4     4     4    1    12     5    12    2     2   
50                        35    10    13    4     1     2     1    7     5   
51                         2     0     3    6    14     7    16   11    23   
55                         0     1     2    2     1     4     0    1     0   
60                         0     1     0    0     0     0     1    0     2   
70                         0     0     0    0     0     1     0    0     0   
71                         0     1     0    0     0     0     2    0     1   
72                         0     0     0    0     5    11     4    2     9   
75                         3     0     0    0     0     0     0    1     4   
81                        46    40    50    7     0     6    12   27    19   
82                         3     1     0    1     2     1     0    0     2   
All                     1576  1025  1056  955  1215  1236  1474  890  1441   

                                  
Year                         All  
Quarter      2     3    4         
Division                          
10          59    34   29    843  
15          98    82   16    483  
18          11    13    0    119  
20           0     0    0     22  
21          13    14    5    160  
22          36    46   24    470  
29           3     1    0     18  
30         888   779  117  10157  
32          52    51   12    162  
33           0     1    0     13  
34          49    43   16    145  
35          82    98   54    419  
40          15    19    2    137  
41           2     1    1     50  
50           6     6    1     91  
51          10     9   10    111  
55           2     0    0     13  
60           0     1    1      6  
70           0     0    0      1  
71           0     2    0      6  
72           1     0    0     32  
75           4    10    2     24  
81          44    43   15    309  
82           3     1    0     14  
All       1378  1254  305  13805

In [138]:
df_complaints_gy = (
    df_complaints
    .query("`If Manufacturing Complaint` == 'Y' and Year.isin([2021,2022,2023])")
    .assign(Quarter =lambda d: d['Notification Created Date'].dt.quarter,
            Division = lambda d: d['Division'].map(str),
            )
    .groupby(['Division','Year','Quarter','Month'],as_index=False).size()
)
df_complaints_gy.rename(columns={'size':'Complaints'},inplace=True)

In [139]:
df_BUOM_2021  = pd.read_excel(r'C:\Medline\2. CPM\4. CPM by BUOM\BUOM Database\history\US_ShipmentData_2021.xlsx', usecols='D,E,AB,AP,BI')
df_BUOM_2022  = pd.read_excel(r'C:\Medline\2. CPM\4. CPM by BUOM\BUOM Database\history\US_ShippingData_2022.xlsx', usecols='D,E,AB,AP,BI')
df_BUOM_2023  = pd.read_excel(r'C:\Medline\2. CPM\4. CPM by BUOM\BUOM Database\2023\US BUOM 202311.xlsx', usecols='C,D,H,P,Q')

In [140]:
df_BUOM_all = pd.concat([df_BUOM_2022,df_BUOM_2023,df_BUOM_2021],ignore_index=True)

In [141]:
df_BUOM_all.dropna(subset=['StrippedVendorNumber'],inplace=True)

In [142]:
df_BUOM_all['Year'] = df_BUOM_all['ShipmentDt'].dt.year
df_BUOM_all['Quarter'] = df_BUOM_all['ShipmentDt'].dt.quarter
df_BUOM_all['Month'] = df_BUOM_all['ShipmentDt'].dt.month

In [143]:
df_BUOM_all.rename(columns={'StrippedVendorNumber':'VendorNumber','ProductDivisionCode':'Division','BUOMShipmentQty':'BUOM'},inplace=True)
df_BUOM_all['Division'] = df_BUOM_all['Division'].map(int).map(str)
df_BUOM_all['VendorNumber'] = df_BUOM_all['VendorNumber'].map(int).map(str)

In [144]:
df_BUOM_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 803048 entries, 0 to 803050
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   VendorNumber  803048 non-null  object        
 1   VendorName    803048 non-null  object        
 2   Division      803048 non-null  object        
 3   BUOM          803048 non-null  int64         
 4   ShipmentDt    803048 non-null  datetime64[ns]
 5   Year          803048 non-null  int64         
 6   Quarter       803048 non-null  int64         
 7   Month         803048 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(3)
memory usage: 55.1+ MB


In [145]:
df_BUOM_all

,VendorNumber,VendorName,Division,BUOM,ShipmentDt,Year,Quarter,Month
0,105110,SINO PROTECTION HOLDINGS CO LTD,21,14400,2022-06-24,2022,2,6
1,105110,SINO PROTECTION HOLDINGS CO LTD,21,7560,2022-06-24,2022,2,6
2,105110,SINO PROTECTION HOLDINGS CO LTD,21,1920,2022-06-24,2022,2,6
3,105110,SINO PROTECTION HOLDINGS CO LTD,21,2688,2022-06-24,2022,2,6
4,105110,SINO PROTECTION HOLDINGS CO LTD,21,896,2022-06-24,2022,2,6
...,...,...,...,...,...,...,...,...
803046,134137,APEX TRANSGULF MANUFACTURING,50,1200000,2021-12-29,2021,4,12
803047,148393,COMBO ZHONGSHAN MEDICAL EQUIPMENT C,30,1904,2021-12-20,2021,4,12
803048,105110,SINO PROTECTION HOLDINGS CO LTD,22,12740,2021-11-23,2021,4,11
803049,105110,SINO PROTECTION HOLDINGS CO LTD,22,1150,2021-11-23,2021,4,11


In [146]:
vendor_mapping_inspection = vendor_mapping.loc[(~vendor_mapping['Regional Manager'].isin(['Exemption','US vendor']))&(vendor_mapping['Regional Manager'].notnull()),'Vendor Number'].to_list()
df_BUOM_all = df_BUOM_all.loc[df_BUOM_all['VendorNumber'].isin(vendor_mapping_inspection)]

In [147]:
df_BUOM_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 775777 entries, 0 to 803050
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   VendorNumber  775777 non-null  object        
 1   VendorName    775777 non-null  object        
 2   Division      775777 non-null  object        
 3   BUOM          775777 non-null  int64         
 4   ShipmentDt    775777 non-null  datetime64[ns]
 5   Year          775777 non-null  int64         
 6   Quarter       775777 non-null  int64         
 7   Month         775777 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(3)
memory usage: 53.3+ MB


In [148]:
df_BUOM_gy = df_BUOM_all.groupby(['Division','Year','Quarter','Month'],as_index=False)['BUOM'].sum()

In [149]:
df_BUOM_gy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 890 entries, 0 to 889
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Division  890 non-null    object
 1   Year      890 non-null    int64 
 2   Quarter   890 non-null    int64 
 3   Month     890 non-null    int64 
 4   BUOM      890 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 34.9+ KB


In [150]:
df_complaints_gy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 479 entries, 0 to 478
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Division    479 non-null    object
 1   Year        479 non-null    int64 
 2   Quarter     479 non-null    int64 
 3   Month       479 non-null    int64 
 4   Complaints  479 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 18.8+ KB


In [151]:
df_cpm = pd.merge(df_complaints_gy,df_BUOM_gy,on=['Division','Year','Quarter','Month'],how='outer').fillna(0)

In [152]:
df_cpm.to_excel('./out.xlsx')

In [153]:
pivot_table  = (
    df_cpm
    .pivot_table(
        index = ['Division'],
        columns = ['Year','Quarter'],
        values = ['Complaints', 'BUOM'], # 指定想要运算的列
        aggfunc = {'Complaints': 'sum', 'BUOM': 'sum'},
        fill_value = 0
    )
)


In [154]:
pivot_table.to_excel('pivot_table.xlsx')